# Read and verify integrity of G file

In [1]:
import sys
sys.path.append('..')
import g_eqdsk.g_file as sgf
import sub.plot as spl
import plotly.express as px
import pandas as pd
import numpy as np

## Read G file and display flux

In [2]:
fileName = '0_g035450.02750'
#pr = sgf.read_g_file('g104362.00213')
pr = sgf.read_g_file(fileName)

In [3]:
spl.contour(pr['psizr'], pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

## Pressure Consistency Check

In [4]:
# Convert normalized flux to flux
def get_flux(normflux):
    return pr['simag']+(pr['sibry']-pr['simag'])*normflux
# Calculate the differential waveform_The number of elements is reduced by one
def get_diff(x, y):
    dy = np.append(y, 0)-np.append(0, y)
    dx = np.append(x, 0)-np.append(0, x)
    v = dy/dx
    v = v[1:-1]
    return np.append(v, v[-1])

In [5]:
# Differentiate the pressure waveform with the flux and confirm that it matches dp/df.
y = np.array(pr['pres'])
x = np.linspace(0, 1, len(y))
x2 = get_flux(x)
df = pd.DataFrame(data=np.array([x2, y]).T, columns=['flux', 'pressure'])
fig2 = px.line(df, x='flux', y='pressure')
fig2.show()
# pprime by computing derivatives
v = get_diff(x2, y)
df = pd.DataFrame(data=np.array([x2, v]).T, columns=['flux', 'calc_dp'])
fig2 = px.line(df, x='flux', y='calc_dp')
fig2.show()
# pprime with g-file
y1 = pr['pprime']
df = pd.DataFrame(data=np.array([x2, y1]).T, columns=['flux', 'pprime'])
fig2 = px.line(df, x='flux', y='pprime')
fig2.show()

In other words, we confirmed that pprime can be calculated from the pressure distribution.  
It can be seen that pprime takes the derivative of the flux, not the normalized flux.

## ポロイダル電流に関する整合性の確認

In [6]:
# fpolの波形をフラックスで微分して、ffprimに一致することを確認
y = np.array(pr['fpol'])
x = np.linspace(0, 1, len(y))
df = pd.DataFrame(data=np.array([x, y]).T, columns=['norm flux', 'fpol'])
fig2 = px.line(df, x='norm flux', y='fpol')
fig2.show()
# 規格化フラックスでのfpol
x2 = get_flux(x)
df = pd.DataFrame(data=np.array([x2, y]).T, columns=['flux', 'fpol'])
fig2 = px.line(df, x='flux', y='fpol')
fig2.show()
# fprimを計算
v = get_diff(x2, y)
df = pd.DataFrame(data=np.array([x2, v]).T, columns=['flux', 'calc_fprim'])
fig2 = px.line(df, x='flux', y='calc_fprim')
fig2.show()
# 計算によるffprim
df = pd.DataFrame(data=np.array([x2, v*y]).T, columns=['flux', 'calc_ffprim'])
fig2 = px.line(df, x='flux', y='calc_ffprim')
fig2.show()
# g-fileからのffprim
y1 = pr['ffprim']
df = pd.DataFrame(data=np.array([x2, y1]).T, columns=['flux', 'ffprim'])
fig2 = px.line(df, x='flux', y='ffprim')
fig2.show()

fpolからffprimが計算できることが確認できた。
これも微分は、正規化フラックスでなく、フラックスから計算している。

## 全電流の確認

### ドメインの作成と確認

In [7]:
# domain matrixの作成と確認
dm = sgf.get_domain(pr, 'BBBS')
spl.heatmap(dm, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

### 全電流を算出

全電流をpprime, ffprimから算出して、全電流に整合するかを確認する。

In [8]:
from scipy import interpolate

# pprimeの線形補間関数の作成
y = pr['pprime']
x = np.linspace(0, 1, len(y))
fnc_pprim = interpolate.interp1d(x, y)

# ffprimの線形補間関数の作成
y = pr['ffprim']
x = np.linspace(0, 1, len(y))
fnc_ffprim = interpolate.interp1d(x, y)


In [9]:
# 正規化フラックスの作成
nf = (pr['psizr']-pr['simag'])/(pr['sibry']-pr['simag'])
nf = nf.reshape(-1)

# 1.0が境界、境界があいまいな場合は1にしておく
# 0.0にしてしまうと、磁気軸の値を参照してしまう場合がある。
nf = np.array([e if 0.0 <= e <= 1.0 else 1.0 for e in nf])
nf = nf.reshape(pr['nh'], pr['nw'])
nf *= dm # ドメインの外の領域は削除

In [10]:
spl.contour(nf, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

In [11]:
# r位置のマトリックスを作成
rm = [np.linspace(pr['rleft'], pr['rleft']+pr['rdim'], pr['nw'])]*pr['nh']
rm = np.array(rm)+10**(-7)

In [12]:
# 電流密度分布の作成 
# ポロイダルカレントをu0で割っていることに注意
jt = rm*fnc_pprim(nf)*dm + fnc_ffprim(nf)/rm*dm/(4*np.pi*1.0e-7)
jt *= pr['dr']*pr['dz']

In [13]:
spl.contour(jt, pr['rleft'], pr['zmin'], pr['dr'], pr['dz'], pr['shbbbs'])

In [14]:
# トータルの電流値とg_fileでの電流値
jt.sum(), pr['current']

(-61401.286742142176, -61407.1393)

## fpolの値の確認

fpolはトロイダルコイル電流を含めたポロイダル電流である。  
従って、最外殻磁気面上でのポロイダル電流は、トロイダルコイル電流によるポロイダル電流に一致することを確認する。

In [15]:
# トロイダル磁場の値と、その位置
pr['bcentr'], pr['rcentr']

(0.2661, 0.600000024)

In [16]:
# b=(b0 r0)/rであってプラズマの外のb0 r0
pr['bcentr']*pr['rcentr']

0.1596600063864

In [17]:
# fpolの最外殻磁気面の位置で、この値になるで、配列の最後の値がこれに一致する。
pr['fpol']

array([0.16375152, 0.16364673, 0.16354256, 0.16343903, 0.16333618,
       0.16323405, 0.16313265, 0.16303203, 0.16293222, 0.16283325,
       0.16273516, 0.16263797, 0.16254172, 0.16244644, 0.16235217,
       0.16225894, 0.16216678, 0.16207573, 0.16198581, 0.16189706,
       0.16180952, 0.16172322, 0.16163819, 0.16155446, 0.16147207,
       0.16139106, 0.16131145, 0.16123328, 0.16115658, 0.16108139,
       0.16100774, 0.16093566, 0.16086519, 0.16079637, 0.16072921,
       0.16066377, 0.16060007, 0.16053814, 0.16047802, 0.16041975,
       0.16036336, 0.16030887, 0.16025633, 0.16020577, 0.16015722,
       0.16011072, 0.16006629, 0.16002398, 0.15998382, 0.15994583,
       0.15991005, 0.15987652, 0.15984527, 0.15981634, 0.15978974,
       0.15976553, 0.15974372, 0.15972436, 0.15970748, 0.15969311,
       0.15968127, 0.15967202, 0.15966536, 0.15966135, 0.15966001])

#### ポロイダルカレントの式

$$
B=\frac{\mu_{0}}{2 \pi R} I
$$
から
$$
I=\frac{2 \pi}{\mu_{0}} R B
$$
g_fileの説明によれば
$$
F=R B_{T}=\frac{\mu_{0}}{2 \pi}I
$$